**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 

Tento notebook je inšpirovaný podobnými analýzami od iných autorov (napr. [Character Networks Visualization for Les Misérables](https://studentwork.prattsi.org/infovis/labs/character-networks-visualization-for-les-miserables/) a [Game of Thrones – Co-occurrence Network of Characters](https://studentwork.prattsi.org/infovis/labs/game-of-thrones-co-occurrence-network-of-characters/)). Zdrojový kód je však pôvodný a – na rozdiel od iných prác – celý proces, vrátane vizualizácie – sa realizuje v Python-e.



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install umap-learn python-louvain textblob
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from collections import defaultdict
from sklearn.preprocessing import minmax_scale, MinMaxScaler
import numpy as np
import pandas as pd
import itertools
import nltk
import re

import community # package python-louvain
import networkx as nx
from umap import UMAP
from textblob import TextBlob

from IPython.display import display
from matplotlib.colors import to_hex
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/424vr9du2f480d9/three_musketeers.txt?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

# We also need some data from the nltk package
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
def refine_tags(tree, words, case_sensitive=True, infuse_text=False, tag=None):
    class NOTHING: pass
    
    if isinstance(words, set):
        words = {it: tag for it in words}
    
    if infuse_text:
        make_item = lambda word, tag: (word, "{}:{}".format(tag, word))
    else:
        make_item = lambda word, tag: (word, tag)
    
    if case_sensitive:
        normalize = lambda x: x
    else:
        normalize = lambda x: x.lower()
        words = {normalize(k): v for k,v in words.items()}
        
    for item in tree:
        if isinstance(item, tuple):
            word, tag = item
            words_tag = words.get(normalize(word), NOTHING())
            
            # not in dict: yield the original item
            if isinstance(words_tag, NOTHING):
                yield item
            # tag is None: do not change the orginal tag
            elif words_tag is None:                
                yield make_item(word, tag)
            # change the tag to words_tag
            else:
                yield make_item(word, words_tag)
        else:
            yield nltk.Tree(item.label(), refine_tags(item, words))

def refine_chunks(chunked_sents, wordset, grammar, case_sensitive=True,
                  infuse_text=False, tag=None):    
    refined = (list(refine_tags(sent, wordset,
                                case_sensitive=case_sensitive,
                                infuse_text=infuse_text, tag=tag))
                    for sent in chunked_sents)
    rechunked = nltk.RegexpParser(grammar).parse_sents(refined)
    return rechunked

def mix_colors(colors):
    return tuple(np.asarray(colors).mean(axis=0))

def compute_cooccurence(entity_occurences, context=15, multi_count=False):
    cooccurence = {}

    for (ent1, occ1), (ent2, occ2) in itertools.combinations(
        entity_occurences.items(), 2
    ):
        num_occ = 0
        jstart = 0
        
        # for our algorithm to work, we need to make sure the occurrence sequences are sorted
        occ1 = sorted(occ1)
        occ2 = sorted(occ2)
      
        for i in range(len(occ1)):
            for j in range(jstart, len(occ2)):
                if occ1[i] >= occ2[j] - context and occ1[i] <= occ2[j] + context:
                    # we have found a co-occurence
                    num_occ += 1
                    
                    if not multi_count:
                        # once we get a match, we increment i and set jstart
                        # j + 1 so that we do not count the same co-occurrence
                        # more than once
                        jstart = j + 1
                        continue
                
                elif occ1[i] < occ2[j] - context:
                    # we continue with the next i, since occ2[j]
                    # will only get larger
                    break
                    
                elif occ1[i] > occ2[j] + context:
                    # occ1[i] will only get larger, so we do not need to
                    # consider this j in future iterations
                    jstart = j + 1

        cooccurence[(ent1, ent2)] = num_occ

    return cooccurence
  
color_list = np.array([
    (0, 150, 117),
    (0, 196, 255),
    (115, 192, 0),
    (255, 85, 132),
    (204, 173, 170),
    (155, 116, 216),
    
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
    (150, 150, 150),
]) / 255.0

## Grafy literárnych postáv založené na počte spoločných výskytov

V tomto notebook-u budeme realizovať zaujímavú analýzu, ktorá spája prístupy z oblasti spracovania prirodzeného jazyka na jednej strane a z oblasti teórie grafov a analýzy sietí na druhej strane. Naším cieľom bude vytvoriť graf literárnych postáv z určitej knihy a následne na ňom vykonať určité analýzy a vizualizovať ich výsledky. "Príbuznosť" postáv vo výslednej sieti bude závisieť od počtu ich spoločných výskytov v rámci toho istého kontextu. Po vytvorení siete ukážeme, ako je možné vypočítať viacero ukazovateľov – ako je napríklad centralita jednotlivých vrcholov, ako detegovať komunity v sieti (skupiny postáv, ktoré spolu nejakým spôsobom súvisia) atď. Tento typ analýzy má dnes mnoho praktických aplikácií: napr. v analýze sociálnych sietí.

Náš celkový postup bude vyzerať takto:

* Identifikovať v texte pomenované entity: v našom prípade knižné postavy.
* Určiť príbuznosť postáv na základe počtu spoločných výskytov v tom istom kontexte.
* Na základe extrahovaných údajov skonštruovať sieť.
* Vykonať analýzu siete a výsledky vizualizovať.
### Načítanie textu

V prvom kroku, samozrejme, načítame text knihy, ktorú budeme analyzovať. Keďže sa jedná len o jednu knihu, objem textových dát nebude príliš veľký a celý naraz sa bez problémov zmestí do operačnej pamäte. V opačnom prípade by sme ho museli rozdeliť na menšie časti a spracúvať postupne.



In [ ]:
with open('data/three_musketeers.txt', 'r', encoding="utf8") as f:
    sample = f.read()

### Predspracovanie textu

V ďalšom kroku text predspracujeme. Najprv môžeme text mierne štandardizovať: napríklad nahradiť špeciálne typy úvodzoviek a inej interpunkcie kánonickými tvarmi a pod. Potom text rozdelíme na jednotlivé vety. Následne sa budú jednotlivé vety tokenizovať, t.j. rozdelia sa na menšie jednotky (slová, slovné spojenia).



In [ ]:
# Špeciálne úvodzovky ‘’ nahradíme klasickými jednoduchými úvodzovkami '.
trans_table = str.maketrans("‘’", "''")
sample = sample.translate(trans_table)

# Text rozdelíme na vety.
sentences = nltk.sent_tokenize(sample)

# Zopár viet vypíšeme ako príkad.
for sent in sentences[15:18]:
    print(sent, "\n")

In [ ]:
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

for tok_sent in tokenized_sentences[15:18]:
    print(tok_sent, "\n")

Ďalej jednotlivé symboly (token-y) otagujeme, čím získame informáciu o ich úlohe vo vete – napríklad či predstavujú podstatné meno, sloveso a pod.



In [ ]:
tagged_sentences = nltk.pos_tag_sents(tokenized_sentences)

for tag_sent in tagged_sentences[15:18]:
    print(tag_sent, "\n")

Ďalej skúsime extrahovať informácie o štruktúre textu pomocou tzv. chunkovania.



In [ ]:
chunked_sentences_sample = nltk.ne_chunk_sents(tagged_sentences[15:18])

for chunked_sent in chunked_sentences_sample:
    print(chunked_sent, "\n")

### Rozpoznávanie entít

Po chunkovaní vieme aplikovať algoritmus na rozpoznávanie pomenovaných entít. Keďže balíček `nltk` realizuje túto úlohu jednou z klasických metód, ktoré nevynikajú perfektnou robustnosťou a aj s ohľadom na charakter každého konkrétneho textu môže byť výsledky potrebné korigovať aj ručne, alebo prípadne upraviť vstupné dáta tak, aby sa dosiahol korektný výsledok.

V každom prípade si budeme potrebovať uložiť aj miesta výskytu jednotlivých entít, aby sme neskôr boli schopní vyrátať, ako často sa entity vyskytovali spoločne v tom istom kontexte.



In [ ]:
# V texte rozpoznáme entity.
chunked_sentences = nltk.ne_chunk_sents(tagged_sentences)

# Ak sa pred osobou vyskytuje titul, vtiahneme ho do mena entity.
titles = {"professor", "madame", "madam", "mr", "mr.", "mrs", "mrs.",
          "miss", "uncle", "aunt", "lord"}
rechunked_sentences = refine_chunks(chunked_sentences, titles, tag="TIT",
                          grammar = r"""
                              PERSON: {<TIT><PERSON>}
                          """,
                          case_sensitive=False)

# Zhromaždíme miesta výskytov jednotlivých entít.
# Paralelne si ukladáme finálnu podobu predspracovaného textu.
entity_occurences = defaultdict(list)
chunk_list = []
num_chunks = 0

for sent_tree in rechunked_sentences:    
    for node in sent_tree:
        # a regular tagged token
        if isinstance(node, tuple):
            chunk_list.append(node[0])
        
        # a sub-tree corresponding to an entity
        else:
            identifier = " ".join((leaf[0] for leaf in node.leaves())).strip()    
            entity_occurences[identifier].append(num_chunks)
            chunk_list.append(identifier)
        
        num_chunks += 1

### Zlučovanie rôznych mien tých istých entít

Nedá očakávať, že proces rozpoznávania entít prebehne plne automaticky. Niektoré postavy sa v knihách označujú viacerými menami: napríklad rôznymi prezývkami, zdrobneninami a pod. Zlúčiť všetky tieto rozličné mená dokopy ako označenia tej istej entity je väčšinou bez skutočného porozumenia textu (ktorým existujúce metódy jednoducho nedisponujú) nemožné, preto budeme musieť aspoň časť tejto práce vykonať ručne.

Nájdené entity si teda zobrazíme, zotriedené podľa počtu výskytov a kde je to potrebné, ručne ich zlúčime. Zlučovanie by bolo možné aj čiastočne automatizovať, napr. pomocou informácií z iného zdroja: môže sa nám napríklad podariť nájsť webovú stránku so zoznamom postáv aj s ich rôznymi menami a prezývkami.



In [ ]:
for k, v in sorted(entity_occurences.items(), key=lambda it: -len(it[1])):
    print("{}x\t{}".format(len(v), k))

Manuálne môžeme entity zlúčiť napríklad pomocou slovníka v tvare:

```
entity_dict = {
    'identifikator_entity': {'Meno Entity', 'Alternativne Meno 1', 'Alternativne Meno 2', ...},

    ...
}
```
Entity, ktoré sa vyskytujú málokrát, môžu byť menej podstatné: podľa uváženia ich môžeme v tejto fáze zo slovníka úplne vypustiť.



In [ ]:
entity_dict = {
    "Athos": {"Athos", "Monsieur Athos"},
    "Milady": {"Milady", "Winter", "MILADY"},
    "Porthos": {"Porthos", "Monsieur Porthos", "PORTHOS"},
    "Aramis": {"Aramis", "Monsieur Aramis", "ARAMIS"},
    "Felton": {"Felton"},
    "Bonacieux": {"Bonacieux", "Madame Bonacieux", "Monsieur Bonacieux"},
    "Treville": {"Treville"},
    "Planchet": {"Planchet"},
    "Buckingham": {"Buckingham"},
    "Grimaud": {"Grimaud"},
    "Bazin": {"Bazin"},
    "Mousqueton": {"Mousqueton"},
    "La Rochelle": {"La Rochelle"},
    "Richelieu": {"Richelieu", "Eminence", "Cardinal", "Monsieur Cardinal"},
    "d'Artagnan": {"Gascon"},
    "Rochefort": {"Rochefort"},
    "de Chevreuse": {"Madame de Chevreuse"},
    "Madame Coquenard": {"Madame Coquenard", "Coquenard"},
    "Monsieur Dessessart": {"Monsieur Dessessart"},
    "Louis XIII": {"Louis XIII", "Louis"},
    "Louis XIV": {"Louis XIV"},
}

Získaný slovník teraz invertujeme (tak, aby mapoval všetky alternatívne mená na ten istý identifikátor postavy).



In [ ]:
reverse_entity_dict = defaultdict(set)

for entity, names in entity_dict.items():
    for name in names:
        reverse_entity_dict[name].update({entity})
        
reverse_entity_dict = dict(reverse_entity_dict)

V prípade potreby môžeme otestovať, či sme na nejaké dôležité entity nezabudli:



In [ ]:
forgotten_entity_occurences = {k: v for k, v in entity_occurences.items() if not k in reverse_entity_dict}

for k, v in sorted(forgotten_entity_occurences.items(), key=lambda it: -len(it[1])):
    print("{}x\t{}".format(len(v), k))

Zoznam výskytov transformujeme pomocou inverzného slovníka entít.



In [ ]:
translated_occurences = defaultdict(list)

for entity, occurences in entity_occurences.items():
    try:
        for translated_entity in reverse_entity_dict[entity]:
            translated_occurences[translated_entity].extend(occurences)
    except KeyError:
        pass

Nájdeme spoločné výskyty entít pomocou preddefinovanej funkcie.



In [ ]:
cooccurence = compute_cooccurence(translated_occurences)

Výsledky uložíme do CSV súborov: osobitne vrcholy (entity) a osobitne hrany (ohodnotené počtom spoločných výskytov).



In [ ]:
with open("output/nodes.csv", "w") as nodes_file:
    nodes_file.write("Id,Label,Occurences\n")
   
    for entity, occurences in translated_occurences.items():
        nodes_file.write("{},{},{}\n".format(entity, entity, len(occurences)))

with open("output/edges.csv", "w") as edges_file:
    edges_file.write("Source,Target,Type,id,weight\n")
   
    for i, ((ent1, ent2), num_cooccur) in enumerate(cooccurence.items()):
        if num_cooccur > 0:
            edges_file.write("{},{},Undirected,{},{}\n".format(ent1, ent2, i, num_cooccur))

### Konštrukcia, analýza a vizualizácia grafu

Ďalej skonštruujeme graf, ktorý vznikol na základe analýzy textu a budeme naň aplikovať metódy analýzy grafov. Všetky tieto kroky budeme realizovať v jazyku Python. Bolo by však rovnako jednoduché načítať CSV súbory, ktoré sme vyššie vytvorili, v nejakom externom nástroji, ako je napr. známy softvér na vizualizáciu grafov [gephi](https://gephi.org/). Zvyšný postup súvisiaci s analýzou a vizualizáciou grafu by sa potom dal realizovať tam.

V našom prípade použijeme na konštrukciu grafu python-ový balíček `networkx`. Do novo vytvoreného grafu pridáme všetky vrcholy uložené v súbore `nodes.csv`. Zaujímajú nás ID vrcholov, mená postáv, ktoré sa s nimi spájajú, a celkový počet výskytov príslušných postáv, preto tieto údaje priložíme ku vrcholom ako dáta:



In [ ]:
G = nx.Graph()

nodes = pd.read_csv("output/nodes.csv")
for nid, (node, label, occurences) in nodes[["Id", "Label", "Occurences"]].iterrows():  
    G.add_node(node, label=label, occurences=occurences)

Ďalej načítame súbor `edges.csv` a pridáme do grafu všetky hrany. Váhy im priradíme na základe počtu spoločných výskytov postáv v texte, ako je zaznamený v CSV súbore:



In [ ]:
edges = pd.read_csv("output/edges.csv")
G.add_weighted_edges_from((
       (src, tgt, w)
       for i, (src, tgt, w) in
       edges[["Source", "Target", "weight"]].iterrows()
))

Ak sú v grafe nejaké vrcholy, ktoré sú izolované (nie sú prepojené hranami s inými vrcholmi), odstránime ich, aby bol graf prehľadnejší. Tiež konvertujeme identifikátory vrcholov na celočíselné, aby sa s nimi ľahšie pracovalo:



In [ ]:
G.remove_nodes_from(list(nx.isolates(G)))
G = nx.convert_node_labels_to_integers(G)

#### Určenie "dôležitosti" vrcholov pomocou page-rank-u

Pri vizualizácii grafov sa ponúka veľa možností, ako do výslednej grafickej reprezentácie zakódovať relevantné informácie. Jedna z možností je využiť veľkosť vrcholov. V našom prípade vypočítame pre každý vrchol indikátor centrality známy ako page-rank. Voľne povedané, page-rank bude indikovať, aký dôležitý je vrchol v rámci grafu. Page-rank-y všetkých vrcholov preškálujeme do rozumného rozsahu a na výsledku založíme veľkosť vrcholov. Ten istý prístup použijeme aj na preškálovanie veľkostí fontov, ktoré sa použijú v popiskoch vrcholov.



In [ ]:
page_rank = nx.pagerank(G)
sizes = [page_rank[node] for node in G.nodes()]
sizes = minmax_scale(sizes, (80, 1000))

font_scaler = MinMaxScaler((12, 20))
font_scaler.fit(np.reshape(sizes, (-1, 1)))

#### Hrúbka a transparentnosť hrán

Hrúbky hrán v grafe určíme jednoducho preškálovaním váh jednotlivých hrán: t.j. počtov spoločných výskytov. Tie isté hodnoty, len v inej škále, použijeme aj na určenie miery transparentnosti jednotlivých hrán. Keď niektoré hrany zobrazíme transparentnejšie, bude graf pôsobiť prehľadnejšie.



In [ ]:
widths = [edge[2] for edge in G.edges(data='weight')]
widths = minmax_scale(widths, (1, 5))
edge_alpha = minmax_scale(widths, (0.3, 1))

#### Detekcia komunít v grafe

Ďalej použijeme detekciu komunít z balíčka `community`, aby sme vrcholy rozdelili podľa komunít. Hrubo povedané, ide o približný ekvivalent zhlukovania pre grafy.



In [ ]:
parts = community.best_partition(G, resolution=1.0)
num_parts = np.max(list(parts.values()))

communities = [[] for i in range(num_parts+1)]
for k, v in parts.items():
    communities[v].append(k)

Zafarbenie vrcholov a hrán určíme podľa komunít, do ktorých patria:



In [ ]:
colors = np.asarray([to_hex(color_list[parts.get(node)]) for node in G.nodes()])
cmap = lambda c: color_list[c]

edge_colors = [to_hex(
    mix_colors([cmap(parts.get(src)), cmap(parts.get(dest))]),
      keep_alpha=True) for src, dest, w in G.edges(data='weight')]

#### 2-rozmerné rozmiestnenie vrcholov

Keďže graf zobrazujeme v 2-rozmernom diagrame, musíme všetkým jeho vrcholom pochopiteľne priradiť nejaké pozície. Ideálne by bolo urobiť to tak, aby sa úzko súvisiace vrcholy nachádzali blízko seba a aby sa hrany vzájomne príliš neprekrývali. Existuje mnoho metód na určenie rozloženia vrcholov. Nie všetky z nich však poskytujú dobré výsledky v prípade, že je graf zložitý a obsahuje veľký počet spojení. Pri určovaní rozloženia vrcholov preto mierne zneužijeme UMAP – metódu pôvodne určenú na znižovanie rozmeru dát – a rozloženie určíme pomocou nej.

Jedným zo vstupov metódy UMAP sú vzdialenosti medzi bodmi. V našom prípade vytvoríme maticu vzdialeností invertovaním váh hrán (počtov spoločných výskytov). UMAP bude potom združovať vrcholy, ktoré sa často vyskytujú v tom istom kontexte.

Aby sme zabezpečili, že budú vrcholy z tej istej komunity držať pokope, zvolíme počiatočné pozície všetkých vrcholov tak, že komunitám sa priradia rovnako vzdialené body pozdĺž obvodu kružnice (všetky vrcholy patriace do tej istej komunity sú na začiatku na tej istej pozícii).



In [ ]:
# set up the distance matrix for UMAP
num_nodes = G.number_of_nodes()
dist_mat = np.zeros([num_nodes, num_nodes])
max_weight = edges["weight"].max()

for n1, n2, w in G.edges(data="weight"):
    invdist = w / max_weight
    dist_mat[n1, n2] += invdist
    dist_mat[n2, n1] += invdist
dist_mat = dist_mat.max() - dist_mat

# set up the initial node positions for UMAP
num_communities = len(communities)
community_angles = np.asarray(
    [ic*2*3.14/num_communities for ic in range(num_communities)])
community_centers = np.stack(
    [np.sin(community_angles) * 100, np.cos(community_angles) * 100], axis=1)

init = np.zeros([num_nodes, 2])
for c, mem in enumerate(communities):
    for n in mem:
        init[n] = community_centers[c]

# run UMAP
umap = UMAP(
    metric='precomputed',
    init=init,
    min_dist=10,
    spread=50,
)

pos = umap.fit_transform(dist_mat)
pos = {ipos: p for ipos, p in enumerate(pos)}

#### Vykreslenie grafu



A nakoniec nezostáva už nič iné než vykresliť samotný graf.



In [ ]:
plt.figure(figsize=[14, 10])

# nodes
nx.draw_networkx_nodes(
     G,
     pos = pos,
     node_size = sizes,
     node_color = colors,
     linewidths = 1,
)

# edges
edge_collection = nx.draw_networkx_edges(
    G,
    pos,
    width = widths,
    edge_color = edge_colors,               
)

# transparency of edges
edge_alpha_colors = [tuple(col[:3]) + (al,) for al, col
                        in zip(edge_alpha, edge_collection.get_colors())]
edge_collection.set_color(edge_alpha_colors)

# edge labels
text_collection = nx.draw_networkx_labels(G, pos,
    labels = {node[0]: node[1].replace(" ", "\n")
                  for node in G.nodes(data="label")})

# font sizes
for node, textobj in text_collection.items():
    rank = np.reshape([page_rank[node]], (1, -1))
    textobj.set_fontsize(font_scaler.transform(rank)[0])

# minor re-styling of the plot
plt.gca().collections[0].set_edgecolor("k")
plt.axis('off')
plt.tight_layout()

## Sentimentálny kontext postáv

Metódy analýzy textu nám umožňujú odhadovať aj sentiment textu (pozitívny, negatívny a pod.). Jednoduché prístupy ku analýze sentimentu implementuje napríklad balíček `TextBlob`. Presnejšie výsledky by bolo možné získať napríklad pomocou niektorej z metód založených na hlbokom učení, ale aj výsledky získané pomocou tejto metódy by mali na hrubú analýzu stačiť.

Skúsme teda extrahovať kontexty, v ktorých sa mená jednotlivých postáv vyskytujú, a určiť ich sentiment. V grafe si potom zobrazíme s akým prevládajúcim sentimentom sa jednotlivé postavy v knihe spájajú.



In [ ]:
# context radius
blob_radius = 10

names = []
sentiments = []
num_occurences = []

# we extract context and accumulate their polarities
for ent, occurences in translated_occurences.items():
    sentiment = 0
    
    for occ in occurences:
        sentiment += TextBlob(" ".join(chunk_list[occ-blob_radius:occ+blob_radius])).polarity
        
    sentiment /= len(occurences)
    
    names.append(ent)
    sentiments.append(sentiment)
    num_occurences.append(len(occurences))

# we track the accumulated sentiment but also the number of occurrences
data = np.asarray([sentiments, num_occurences]).transpose()
entity_sentiments = pd.DataFrame(data, columns=["sentiment", "occurences"], index=names)

Vo výslednom grafe zobrazíme len entity, ktoré sa vyskytujú väčší počet krát. Entity s menším počtom výskytov okrem toho budeme vizualizovať transparentnejšou farbou, aby sme ich odlíšili.



In [ ]:
s = entity_sentiments[entity_sentiments["occurences"] > 25]
s_occ = minmax_scale(np.log(s["occurences"].values), (0.3, 1.0))
s_vals = s["sentiment"].values
s_index = s["sentiment"].index

abs_max = np.abs(s_vals).max()
norm = plt.Normalize(vmin=-abs_max, vmax=abs_max)

plt.figure(figsize=(12, 6))
plt.bar(range(len(s_vals)), s_vals,
  color=[(0, 0, 1, occ) if val >= 0 else 
         (1, 0, 0, occ) for val, occ in zip(s_vals, s_occ)]
)

plt.xticks(range(len(s_vals)), s_index, rotation=90)
plt.ylabel("sentiment")

plt.subplots_adjust(left=0.12, right=0.9, top=0.9, bottom=0.3)
plt.grid(linestyle='--')
plt.gca().set_axisbelow(True)